# AI2
## This is project No.2 and it's a genetics algorithm to decrypt a an encoded text

### Introduction
We have two text file named `encoded_text.txt` and `global_text.txt` and we want to decrypt the encoded text by key. We don't have key and the main text. We want to find the key by using global text and finally find the main text by the key that is found by genetics algorithm and global text.

### Description
The cryptography used in this project is done using an encryption key and a table. To do this, a custom encryption key is first selected. A new string is then created using this key, and at the end, encrypted text is created using the encryption table.
To start the encryption operation, a custom key is first selected and this key is repeated to the number of letters of the text that we want to encrypt. For example, if the text we want to encrypt is "WE ARE DISCOVERED SAVE YOURSELF" and the selected key is the word "RUN", the initial string that is generated will be in the next step of encryption with
The table is used according to the following phrase:<br/>
"RU NRU NRUNRUNRUN RUNR UNRUNRUN" This generated phrase is then converted to the master code string using the encryption table. This phrase is also called the key.<br/>
The cryptographic table used in this project includes the letters of the English alphabet, which are written twice in different rows, and each alphabet is rotated one unit to the left compared to the alphabet of the previous row. For example, when B shifts to the first position in the second row, the letter A is moved to the end of the second row. Now, to convert each letter of the string calculated in the previous step to a password, the column that corresponds to the letter in the main message and the row that corresponds to the letter in the key must be selected from the intersection of the two, The password is obtained.<br/>
The encryption key in this project has 14 letters.

## Section Zero: Clear data and create dictionary
In the first step, we need to process the reference text given(`global_text.txt`) and extract a dictionary from it so that we can use it in later sections to calculate the chromosome fit score. One step in processing a data set is to clean the data. For this step as you can see in `clean_text()` function:
<ul>
    <li>
        first we make a list by the characters in global_text.txt and save in clean_text variable
    </li>   
    <li>
        second we replace all characters expect alphabet characters by a space
    </li>   
    <li>
        third replace additional spaces by one space
    </li>   
    <li>
        finally join the words and make a string of them and return its set that will be our dictionary to be used to find the main text
    </li>           
</ul>
Also we have a clean_encoded_text() to clean the encoded text and delete additional letters like symbols and additional spaces that is used to find the key

## Part One: Defining Basic Concepts
In genetic algorithms, we must first provide a definition for the `gene` and then use it to create a `chromosome`. Each chromosome is a set of genes, and this chromosome is a suggested way to solve the problem.<br/>
we define chromosome a word with 14 letters and each letter is a gene that are selected by random library with ALPHABET variable that is a to z letters<br/>

## Part Two: Primary Population Production
After defining and implementing the construction of a chromosome, you must create the first population of chromosomes in a completely random way and without any special orientation. The number of this population can be a problem-solving parameter and depends on your choices.<br/>
We do this part in `generate_initial_population()` function. It selects chromosomes in number `POPULATION_NUM` randomly and save in self.population. Each chromosome has `CONSTANT` genes.

## Part three: Implementing and Specifying the Compatibility Fitness Function
After generating the initial population, we need to define a standard function that can be used to identify superior chromosomes in terms of providing the conditions and constraints of the problem. In this problem, to calculate the chromosome fitness score, we use the dictionary created in section zero.<br/>
It is implemented in `determine_fitness()` function. For each chromosome, it calculates the decoded text's character by this statement:<br/>
`chr((((ord(char) - ord('a')) - (ord(choromosome[gene_index]) - ord('a'))) % len(ALPHABET)) + ord('a'))`<br/>
char in the character in clean_encodedText<br/>
Then it checks that there is a chromosome that have all words in dictionary or not. If it has, we found the solution

## Part Four: Implement Crossover and Mutation and Generate the Next Population
Now, in order to get closer to the decoding key of the problem, we need to create a new population from the populations of our previous generation. We do this with the famous methods that exist in the genetic algorithm, namely mutation and crossover. The crossover function is applied to two chromosomes and combines them to get better chromosomes. The mutation function is applied to a chromosome, using a method to mutate and modify it in the hope that it can achieve a better chromosome.<br/>
We see this in `cross_over_fun()` and `mutation_fun()`. In this part, first we do two point crossover and combine the parents to make children and add them to `new_population` with `CROSS_OVER_PROBABILITY` probability. Then we apply mutation `MUTATION_PROBABILITY` probability for each gene of each chromosomes

## Part Five: Creating a Genetic Algorithm on the Problem
We can see the solution below that use the parts above

### Define constants
Import `random` library because genetics algorithm is associate to stotachic variables<br/>
Import `time` library to estimate the time has been spent

In [4]:
from random import sample, randint, random
from time import time

### Define constants
`CONSTANT` is gene's number<br/>
`SAMPLE_NUM` is the number that has been picked from the population based on their fitness number and do crossover function on them

In [5]:
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
CAPITAL_ALPHABET = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
                    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

CONSTANT = 14
POPULATION_NUM = 60
SAMPLE_NUM = int(POPULATION_NUM / 5)
CROSS_OVER_PROBABILITY = 0.65
MUTATION_PROBABILITY = 0.1


### Define Decoder class
In this class, we implement the parts that has been said before

In [10]:

class Decoder:

    # Part 1
    def __init__(self, globalText, encodedText, keyLength):
        self.globalText = globalText
        self.encodedText = encodedText
        self.decodedText = ""
        self.clean_encodedText = ""
        self.key = ""
        self.keyLength = keyLength
        self.dictionary = set()
        self.population = list()
        self.has_finished = False
        self.fitness = [0] * POPULATION_NUM

    def clean_text(self, text):
        clean_text = list(text)
        for index, char in enumerate(clean_text):
            if char not in ALPHABET and char not in CAPITAL_ALPHABET:
                clean_text[index] = ' '

        clean_text = ''.join(clean_text)
        clean_text = clean_text.lower()

        spilited_clean_text = clean_text.split()
        clean_text = []
        for index, word in enumerate(spilited_clean_text):
            if len(word) > 0:
                clean_text.append(word)

        return set(clean_text)

    def generate_initial_population(self):
        for index in range(POPULATION_NUM):
            word = sample(ALPHABET, self.keyLength)
            while word in self.population:
                word = sample(ALPHABET, self.keyLength)
            self.population.append(word)

    def clean_encoded_text(self):
        self.clean_encodedText = list(self.encodedText)
        for index, char in enumerate(self.clean_encodedText):
            if char not in ALPHABET and char not in CAPITAL_ALPHABET:
                self.clean_encodedText[index] = ' '

        self.clean_encodedText = ''.join(self.clean_encodedText)
        self.clean_encodedText = self.clean_encodedText.split()
        self.clean_encodedText = ' '.join(self.clean_encodedText)

    def decode_text(self, chromosome, is_final_result, text):
        decoded_text = ""
        gene_index = 0

        for char in text:
            if char in ALPHABET:
                decoded_text += chr((((ord(char) - ord('a')) - (
                    ord(chromosome[gene_index]) - ord('a'))) % len(ALPHABET)) + ord('a'))
            elif char in CAPITAL_ALPHABET:
                if not is_final_result:
                    char = char.lower()
                    decoded_text += chr((((ord(char) - ord('a')) - (
                        ord(chromosome[gene_index]) - ord('a'))) % len(ALPHABET)) + ord('a'))
                else:
                    decoded_text += chr((((ord(char) - ord('A')) - (
                        ord(chromosome[gene_index].upper()) - ord('A'))) % len(CAPITAL_ALPHABET)) + ord('A'))
            elif char == ' ':
                decoded_text += ' '
                continue
            elif is_final_result:
                decoded_text += char
                continue

            if gene_index == CONSTANT - 1:
                gene_index = 0
            else:
                gene_index += 1
        return decoded_text

    def determine_fitness(self):

        for index, chromosome in enumerate(self.population):
            self.fitness[index] = 0

            decoded_text = self.decode_text(
                chromosome, False, self.clean_encodedText)

            # Determine new fitness value
            self.has_finished = True
            decoded_text = decoded_text.split()
            for word in decoded_text:
                if word in self.dictionary:
                    self.fitness[index] += 1
                else:
                    self.has_finished = False

            if self.has_finished:
                self.decodedText = self.decode_text(
                    chromosome, True, self.encodedText)
                self.key = chromosome
                break

    def cross_over_fun(self):
        zip_population = {
            tuple(self.population[i]): self.fitness[i] for i in range(len(self.population))}
        zip_population = list(
            reversed(sorted(zip_population.items(), key=lambda x: x[1])))
        print("The chromosome with highest fitness in the current population:")
        print(zip_population[0][1])
        print(''.join(zip_population[0][0]))
        print("\n")
        zip_population = [list(i[0]) for i in zip_population]

        sample_population = []
        for index in range(SAMPLE_NUM):
            sample_population.append(zip_population[index])

        new_population = []
        while len(new_population) != POPULATION_NUM:
            parents = sample(sample_population, 2)
            while parents[0] == parents[1]:
                parents = sample(sample_population, 2)

            child1 = []
            child2 = []

            point1 = randint(1, CONSTANT-2)
            point2 = randint(1, CONSTANT-2)
            while point1 == point2:
                point2 = randint(1, CONSTANT-2)
            if point1 > point2:
                point1, point2 = point2, point1

            flip_coin = sample([0, 1], 1)
            if flip_coin == 1:
                child1 = parents[0][:point1] + \
                    parents[1][point1:point2] + parents[0][point2:]
                child2 = parents[1][:point1] + \
                    parents[0][point1:point2] + parents[1][point2:]
            else:
                child2 = parents[0][:point1] + \
                    parents[1][point1:point2] + parents[0][point2:]
                child1 = parents[1][:point1] + \
                    parents[0][point1:point2] + parents[1][point2:]

            if random() < CROSS_OVER_PROBABILITY:
                if len(new_population) == POPULATION_NUM:
                    break
                else:
                    new_population.append(child1)
                if len(new_population) == POPULATION_NUM:
                    break
                else:
                    new_population.append(child2)

        self.population = new_population

    def mutation_fun(self):
        for index, chromosome in enumerate(self.population):
            for j in range(CONSTANT):
                if random() < MUTATION_PROBABILITY:
                    self.population[index][j] = sample(ALPHABET, 1)[0]

    def decode(self):
        # Part 0
        self.dictionary = self.clean_text(self.globalText)
        self.clean_encoded_text()

        # Part 2
        self.generate_initial_population()

        while not self.has_finished:
            # Part 3
            self.determine_fitness()
            if self.has_finished:
                break
            # Part 4
            self.cross_over_fun()
            self.mutation_fun()
        
        print("------------------------------------------------------")
        print("The key is " + ''.join(self.key))
        print("------------------------------------------------------")
        print("Decoded text:")
        print(''.join(self.decodedText))


In [11]:
encodedText = open('encoded_text.txt').read()
globalText = open('global_text.txt').read()
start = time()
d = Decoder(globalText, encodedText, keyLength=CONSTANT)
decodedText = d.decode()
finish = time()
print("Spended time: " + str(finish - start) + " seconds")

The chromosome with highest fitness in the current population:
41
agyehzlsqtwkiv


The chromosome with highest fitness in the current population:
43
agyehjqintwkiv


The chromosome with highest fitness in the current population:
47
agastwkiftwkav


The chromosome with highest fitness in the current population:
51
agastwkiftwkiv


The chromosome with highest fitness in the current population:
54
aaaeajkiftwkiv


The chromosome with highest fitness in the current population:
54
agastteiktwkic


The chromosome with highest fitness in the current population:
66
agxvapeiasakin


The chromosome with highest fitness in the current population:
64
agxrapeiasakin


The chromosome with highest fitness in the current population:
71
afpeateimtakin


The chromosome with highest fitness in the current population:
75
akarateiatakin


The chromosome with highest fitness in the current population:
81
acxeateimugein


The chromosome with highest fitness in the current population:
83
acprateimugein


The 

The chromosome with highest fitness in the current population:
358
albfrteinstein


------------------------------------------------------
The key is alberteinstein
------------------------------------------------------
Decoded text:
Albert Einstein
Old Grove Rd.
Nassau Point
Peconic, Long Island

August 2nd, 1939

F.D. Roosevelt,
President of the United States,
White House
Washington, D.C.

Sir:

Some recent work by E. Fermi and L. Szilard, which has been communicated to me in manuscript, leads me to expect that the element uranium may be turned into a new and important source of energy in the immediate future. Certain aspects of the situation which has arisen seem to call for watchfulness and, if necessary, quick action on the part of the Administration. I believe therefore that it is my duty to bring to your attention the following facts and recommendations:

In the course of the last four months it has been made probableâ€”through the work of Joliot in France as well as Fermi and S

## Part six: Questions
### Q1: What problems do very small or very large initial populations cause?
If the population size is too large, it takes too much time for each population to perform various operations such as crossover, mutation, etc and in fact the efficiency of the algorithm decreases and creates problems for fitness.<br/>
If the population is very small, our state space is very limited, and in fact the amount we initially quantify is limited to the same space, and the choises will be limited to much to initialized population at first step

### Q2: If the number of population increases in each period, what effect will it have on the accuracy and speed of the algorithm?
By doing this, the further we go, we have to calculate the fitness for more chromosomes, and the algorithm takes more time, the more it goes, we get close to better answers, and our fitness is likely to get better. Making a new chromosome is likely to be added with low fitness, which according to the algorithm either does not affect the final answer or causes it to move away from the final answer.

### Q3: Why are both crossovers and mutations used in genetic algorithms? What problem might occur if any of them are not used?
In the genetic algorithm for crossover operation, it is used to combine different answers to reach new answers that are derived from the previous answers. Mutation operation is used to change different genes in different chromosomes to achieve new and random genes from the domain and make small changes in chromosomes so that we do not get stuck in certain genes. If there is no crossover operation, the previous answers are repeated continuously, and only mutation can not make a lot of change and we do not get new answers. If we do not use mutauion, we may get stuck in a range of genes and fail to reach final answer.

### Q4: Which crossover and mutation operations are more effective in increasing accuracy? Which one makes this impact faster?
I think crossover is more effective in increasing accuracy and it impacts faster in this property.

### Q5: with using crossover and mutation methods, the population may still not change after a few steps. How can this problem be eliminated?
We can increase MUTATION_PROBABILITY and CROSSOVER_PROBABILITY to increase the state space

### Q6: If only one of the crossover and mutation methods were to be used, which one do you think would be more effective? Why?
I think crossover is more effective. Because it make new chromosomes and we can make uniform crossover that the change's rate will be higher than mutation

### Q7: What do you think are the solutions to get a quick answer to this particular problem?
We can change the hyperparameters like `POPULATION_NUM`, `SAMPLE_NUM`, `MUTATION_PROBABILITY` and `CROSS_OVER_PROBABILITY`. We can use different values for these variables to find a faster way